# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
api_url = os.getenv("OPENROUTER_API_BASE")

# Check the key

if not api_key:
    print(
        "No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!"
    )
elif not api_key.startswith("sk-"):
    print(
        "An API key was found, but it doesn't start sk-; please check you're using the right key - see troubleshooting notebook"
    )
elif api_key.strip() != api_key:
    print(
        "An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook"
    )
else:
    print("API key found and looks good so far!")

MODEL = 'openai/gpt-5-nano'
openai = OpenAI(api_key=api_key, base_url=api_url)

API key found and looks good so far!


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = {
    "role": "system", 
    "content": """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""
}

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return {"role": "user", "content": user_prompt}

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))

{'role': 'user', 'content': '\nHere is the list of links on the website https://edwarddonner.com -\nPlease decide which of these are relevant web links for a brochure about the company, \nrespond with the full https URL in JSON format.\nDo not include Terms of Service, Privacy, email links.\n\nLinks (some might be relative links):\n\nhttps://edwarddonner.com/\nhttps://edwarddonner.com/curriculum/\nhttps://edwarddonner.com/proficient/\nhttps://edwarddonner.com/connect-four/\nhttps://edwarddonner.com/outsmart/\nhttps://edwarddonner.com/about-me-and-about-nebula/\nhttps://edwarddonner.com/posts/\nhttps://edwarddonner.com/\nhttps://news.ycombinator.com\nhttps://nebula.io/?utm_source=ed&utm_medium=referral\nhttps://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html\nhttps://edwarddonner.com/curriculum/\nhttps://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/\nhttps://edwarddonner.com/2026/01/04/a

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[link_system_prompt, get_links_user_prompt(url)],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[link_system_prompt, get_links_user_prompt(url)],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling openai/gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'careers page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [13]:
brochure_system_prompt = {
    "role": "system", 
    "content": """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""
}

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = {
#     "role": "system", 
#     "content": """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return {"role": "user", "content": user_prompt}

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling openai/gpt-5-nano
Found 6 relevant links


{'role': 'user',
 'content': '\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-OCR\nUpdated\n4 days ago\n•\n149k\n•\n739\nmoonshotai/Kimi-K2.5\nUpdated\n2 days ago\n•\n274k\n•\n1.79k\nQwen/Qwen3-Coder-Next\nUpdated\n3 days ago\n•\n34.9k\n•\n533\nstepfun-ai/Step-3.5-Flash\nUpdated\nabout 17 hours ago\n•\n10.9k\n•\n490\ncirclestone-labs/Anima\nUpdated\n6 days ago\n•\n50.8k\n•\n481\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFe

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="openai/gpt-4.1-mini",
        messages=[
            brochure_system_prompt,
            get_brochure_user_prompt(company_name, url)
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling openai/gpt-5-nano
Found 5 relevant links


# Hugging Face – The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a pioneering software development company focusing on **machine learning, natural language processing (NLP), and deep learning** technologies. Founded in 2016 and headquartered in Paris, France, the company is on a mission to **solve and democratize artificial intelligence through natural language**, making advanced AI accessible and collaborative for developers and enterprises worldwide.

With a team of 51-200 passionate employees, Hugging Face builds and maintains a vibrant AI community platform that fosters collaboration, discovery, and innovation in the field of machine learning.

---

## Our Platform

### The Home of Machine Learning

Hugging Face offers a comprehensive platform enabling the machine learning community to:

- Collaborate on over **2 million models**, **500,000 datasets**, and **1 million applications** across various AI modalities, including text, image, video, audio, and 3D.
- Host and manage **unlimited public models, datasets, and applications**, accelerating innovation and model deployment.
- Access an open-source AI stack designed to help developers move faster in building machine learning solutions.
- Create portfolios and build profiles to showcase their work to the global ML community.

### Key Features

- **Models & Datasets**: Browse trending AI models and datasets updated frequently by the community.
- **Spaces**: Run AI demos with features like text-to-speech transformation, music generation, image creation from text, and animation.
- **Enterprise-grade Solutions**: Secure, scalable compute and tooling solutions tailored for teams and enterprises.

Explore the platform at [https://huggingface.co](https://huggingface.co) to join the community or to access leading-edge AI applications today.

---

## Company Culture & Community

Hugging Face promotes a culture of **openness, collaboration, and continuous innovation**. They maintain a constant stream of updates, encouraging users to contribute through benchmark datasets, evaluation results, and open-source model repositories. Their platform acts as a hub where individuals and teams share their AI projects, enhancing reproducibility and transparency in AI research.

Members are empowered to experiment, communicate insights, and develop state-of-the-art AI systems collectively, making Hugging Face not only a company but a global community advancing the future of artificial intelligence.

---

## Customers & Partners

Hugging Face serves a diverse clientele including:

- AI researchers and hobbyists building advanced AI models.
- Enterprise teams requiring secure and scalable AI infrastructure.
- Developers creating applications powered by NLP and other machine learning technologies.

The platform’s enterprise solutions offer tools with **security, access controls, and collaboration features** designed for organizational needs, helping accelerate AI adoption at scale.

---

## Careers at Hugging Face

Hugging Face is growing and seeking talented individuals passionate about AI and machine learning. Current openings span various roles in software development, AI research, product management, and more.

- Join a company dedicated to **open collaboration and democratized AI**.
- Be part of a **fast-moving, innovation-driven environment** shaping the future of technology.
- Work alongside experts passionate about leveraging AI for real-world impact.

Explore career opportunities and apply at the [Hugging Face Careers Page](https://huggingface.co/careers).

---

## Connect and Follow

- Website: [huggingface.co](https://huggingface.co)  
- LinkedIn: [Hugging Face on LinkedIn](https://www.linkedin.com/company/huggingface)  
- GitHub: [Hugging Face GitHub](https://github.com/huggingface)  

**Join the AI community building the future with Hugging Face.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="openai/gpt-4.1-mini",
        messages=[
            brochure_system_prompt,
            get_brochure_user_prompt(company_name, url)
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>